<a href="https://colab.research.google.com/github/rittikcodes/fake_news_detection/blob/main/test_fake_news_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
data_fake = pd.read_csv("/content/gdrive/MyDrive/fake_news_detection/Fake.csv")
data_true = pd.read_csv("/content/gdrive/MyDrive/fake_news_detection/True.csv")
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
# assigning fake news as 0 and true news as 1
data_fake["class"] = 0
data_true["class"] = 1

In [6]:
data_true.shape

(21417, 5)

In [7]:
data_fake.shape

(23481, 5)

In [8]:
# taking 10 rows from the dataset for preforming manual testing and removing these 10 data from the original dataset
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis = 0, inplace = True) # axis = 0 => row-wise

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis = 0, inplace = True) # axis = 0 => row-wise

In [9]:
# data_fake_manual_testing["class"] = 0
# data_true_manual_testing["class"] = 1

# <ipython-input-9-a26436eb6887>:1: SettingWithCopyWarning:
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead

# See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
#   data_fake_manual_testing["class"] = 0
# <ipython-input-9-a26436eb6887>:2: SettingWithCopyWarning:
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead

# See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
#   data_true_manual_testing["class"] = 1

<ipython-input-9-a26436eb6887>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"] = 0
<ipython-input-9-a26436eb6887>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"] = 1


In [10]:
# For data_fake_manual_testing
data_fake_manual_testing.loc[:, "class"] = 0

# For data_true_manual_testing
data_true_manual_testing.loc[:, "class"] = 1

In [11]:
data_fake_manual_testing

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [12]:
data_true_manual_testing

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [13]:
data_merge = pd.concat([data_fake, data_true], axis = 0) # axis = 0 => row-wise
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
data = data_merge.drop(["title", "subject", "date"], axis = 1) # axis = 1 => column-wise

In [15]:
def wordopt(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\\W', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w\d\w*', '', text)
    return text

In [16]:
data["text"] = data["text"].apply(wordopt)

In [17]:
X = data["text"]
y = data["class"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
Xv_train = vectorization.fit_transform(X_train)
Xv_test = vectorization.transform(X_test)

In [20]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(Xv_train, y_train)

LogisticRegression()

In [21]:
pred_lr = LR.predict(Xv_test)
LR.score(Xv_test, y_test)
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5881
           1       0.99      0.99      0.99      5339

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# **2nd Model - Decesion Tree**

In [22]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(Xv_train, y_train)

DecisionTreeClassifier()

In [23]:
pred_dt = DT.predict(Xv_test)
DT.score(Xv_test, y_test)
print(classification_report (y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5881
           1       1.00      1.00      1.00      5339

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [24]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    return print("\n\nLR Prediction: {}\nDT Prediction: {}".format(output_label(pred_LR[0]), output_label(pred_DT[0])))

In [25]:
n1 = data_fake_manual_testing["text"].head(1)
print(n1)

23471    21st Century Wire says This week, the historic...
Name: text, dtype: object


In [26]:
news = str(n1)
manual_testing(news)



LR Prediction: Fake News
DT Prediction: Fake News


In [ ]:
n2 = data_true_manual_testing["text"].head(1)
print(n2)

21407    SAO PAULO (Reuters) - Cesar Mata Pires, the ow...
Name: text, dtype: object


In [ ]:
news = str(n2)
manual_testing(news)



LR Prediction: Not A Fake News
DT Prediction: Not A Fake News


In [ ]:
df = pd.read_csv("news.csv")

In [ ]:
df.shape

(6335, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [ ]:
df.describe(include = "all")

,Unnamed: 0,title,text,label
count,6335.000000,6335,6335,6335
unique,NaN,6256,6060,2
top,NaN,OnPolitics | 's politics blog,"Killing Obama administration rules, dismantlin...",REAL
freq,NaN,5,58,3171
mean,5280.415627,NaN,NaN,NaN
std,3038.503953,NaN,NaN,NaN
min,2.000000,NaN,NaN,NaN
25%,2674.500000,NaN,NaN,NaN
50%,5271.000000,NaN,NaN,NaN
75%,7901.000000,NaN,NaN,NaN


In [ ]:
df["text"] = df["text"].apply(wordopt)

In [ ]:
df.head(20)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,daniel greenfield a shillman journalism fellow...,FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,us secretary of state john f kerry said monday...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,— kaydee king kaydeeking november 9 the lesso...,FAKE
4,875,The Battle of New York: Why This Primary Matters,its primary day in new york and frontrunners h...,REAL
5,6903,"Tehran, USA",i’m not an immigrant but my grandparents are...,FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,share this baylee luciani left screenshot of w...,FAKE
7,95,‘Britain’s Schindler’ Dies at 106,a czech stockbroker who saved more than jewis...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,hillary clinton and donald trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,iranian negotiators reportedly have made a las...,REAL


In [ ]:
data.head(20)

,text,class
0,donald trump just couldn t wish all americans ...,0
1,house intelligence committee chairman devin nu...,0
2,on friday it was revealed that former milwauke...,0
3,on christmas day donald trump announced that h...,0
4,pope francis used his annual christmas day mes...,0
5,the number of cases of cops brutalizing and ki...,0
6,donald trump spent a good portion of his day a...,0
7,in the wake of yet another court decision that...,0
8,many people have raised the alarm regarding th...,0
9,just when you might have thought we d get a br...,0


In [ ]:
# data_merge

# Find duplicate values
duplicates = []
for value in data["text"]:
    if value in df["text"].values:
        if value != " ":
            duplicates.append(value)

# Display duplicate values
print("Duplicate values found:")
print(duplicates)


Duplicate values found:
['  ', '  ', '  ', '  ', '  ', '  ']


In [ ]:
n3 = df["text"].head(1)
print(n3)

0    daniel greenfield a shillman journalism fellow...
Name: text, dtype: object


In [ ]:
news = str(n3)
manual_testing(news)



LR Prediction: Fake News
DT Prediction: Fake News


In [ ]:
df.head(1)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,daniel greenfield a shillman journalism fellow...,FAKE


In [ ]:
df[:20]["text"]

0     daniel greenfield a shillman journalism fellow...
1     google pinterest digg linkedin reddit stumbleu...
2     us secretary of state john f kerry said monday...
3     — kaydee king kaydeeking november 9  the lesso...
4     its primary day in new york and frontrunners h...
5       i’m not an immigrant but my grandparents are...
6     share this baylee luciani left screenshot of w...
7     a czech stockbroker who saved more than  jewis...
8     hillary clinton and donald trump made some ina...
9     iranian negotiators reportedly have made a las...
10    cedar rapids iowa — “i had one of the most won...
11    donald trump’s organizational problems have go...
12    click here to learn more about alexandras pers...
13    october   at  am pretty factual except for wom...
14    killing obama administration rules dismantling...
15    as more women move into high offices they ofte...
16    shocking michele obama  hillary caught glamori...
17    0 hillary clinton has barely just lost the

In [ ]:
for i in df[:20]["text"]:
    news = str(i)
    res = manual_testing(news)
    print("***************************************")



LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Not A Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Not A Fake News
DT Prediction: Fake News
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
****************

In [ ]:
for index, row in df[:20].iterrows():
    news = str(row["text"])
    res = manual_testing(news)
    print("Actual: ", row["label"])
    print("***************************************")



LR Prediction: Fake News
DT Prediction: Fake News
Actual:  FAKE
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  FAKE
***************************************


LR Prediction: Not A Fake News
DT Prediction: Fake News
Actual:  REAL
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  FAKE
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  REAL
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  FAKE
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  FAKE
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  REAL
***************************************


LR Prediction: Fake News
DT Prediction: Fake News
Actual:  REAL
***************************************


LR Prediction: Not A Fake News
DT Pred

In [ ]:
json_data = df = pd.read_json("cnn_data.json")
json_data.shape

(3500, 12)

In [ ]:
n4 = df["content"].head(1)

In [ ]:
news = str(n4)
manual_testing(news)



LR Prediction: Fake News
DT Prediction: Fake News


In [ ]:
news = str(n1)
manual_testing(news)

In [27]:
str1 = "Shahrukh Khan is dead."
news = str(str1)
manual_testing(news)



LR Prediction: Fake News
DT Prediction: Fake News


In [28]:
str2 = """VERY IMPORTANT

When you reach the polling booth and find that your name is not in the voter's list, DO NOT HESITATE !! Just show your Aadhar card or voter ID and ask for "challenge vote" under section 49P and insist on casting your vote.

If you find that someone has already cast your vote, then ask for "tender vote" and do cast your vote. Don't just go away.

If any polling booth records more than 14% tender votes, repolling will be conducted in such poling booths.

Please share this very important message with all your friends, as everyone should be aware of their right to vote."""
news = str(str2)
manual_testing(news)



LR Prediction: Fake News
DT Prediction: Fake News
